In [5]:
import pandas as pd
import os
import glob
import re

In [ ]:

# data 폴더 경로 설정
data_folder = 'data'

# data 폴더 내의 모든 CSV 파일 찾기
csv_files = glob.glob(os.path.join(data_folder, '*.csv'))

print(f"발견된 CSV 파일 수: {len(csv_files)}")
print("파일 목록:")
for file in csv_files:
    print(f"  - {os.path.basename(file)}")

# 모든 CSV 파일을 읽어서 딕셔너리에 저장
dataframes = {}

for csv_file in csv_files:
    filename = os.path.basename(csv_file)
    try:
        # CSV 파일 읽기 (인코딩 문제가 있을 수 있으므로 utf-8로 시도)
        df = pd.read_csv(csv_file, encoding='utf-8')
        dataframes[filename] = df
        print(f"\n{filename} 읽기 완료:")
        print(f"  - 행 수: {len(df)}")
        print(f"  - 열 수: {len(df.columns)}")
        print(f"  - 열 이름: {list(df.columns)}")
    except UnicodeDecodeError:
        # utf-8로 읽기 실패시 cp949로 시도
        try:
            df = pd.read_csv(csv_file, encoding='cp949')
            dataframes[filename] = df
            print(f"\n{filename} 읽기 완료 (cp949 인코딩):")
            print(f"  - 행 수: {len(df)}")
            print(f"  - 열 수: {len(df.columns)}")
            print(f"  - 열 이름: {list(df.columns)}")
        except Exception as e:
            print(f"\n{filename} 읽기 실패: {e}")

print(f"\n총 {len(dataframes)}개 파일을 성공적으로 읽었습니다.")

# 첫 번째 파일의 처음 몇 행 확인
if dataframes:
    first_file = list(dataframes.keys())[0]
    print(f"\n{first_file}의 처음 5행:")
    print(dataframes[first_file].head())



In [ ]:
# dataframes의 모든 데이터프레임을 세로로 합치기
import pandas as pd

df_union = pd.concat(dataframes.values(), ignore_index=True)
# 컬럼명에 '$'가 포함된 경우, '$'를 제거하여 컬럼명을 수정
df_union.columns = df_union.columns.str.replace('$', '', regex=False)
df_union['event_time'] = pd.to_datetime(df_union['event_time'])

df_union

In [56]:
df = df_union[['event_time', 'user_id']]
df = df.groupby('user_id').agg(event_time_count=('event_time', 'count'),
                         event_time_min=('event_time', 'min'),
                         event_time_max=('event_time', 'max')).reset_index()

# user_id 컬럼에 float 형태의 문자열('1.0' 등)이 포함되어 int로 변환이 실패함
# 소수점 이하가 0인 경우 int로 변환, 그렇지 않으면 에러 발생
def safe_int_convert(x):
    try:
        f = float(x)
        if f.is_integer():
            return int(f)
        else:
            return x  # 변환 불가한 값은 그대로 반환
    except:
        return x

df['user_id'] = df['user_id'].apply(safe_int_convert)

df.query('user_id == "nan"')

,user_id,event_time_count,event_time_min,event_time_max
35899,nan,2,2025-07-14 22:33:11,2025-07-14 22:33:12


In [61]:
df.sort_values(by='event_time_max', ascending=False)
df.sort_values(by='event_time_min', ascending=True)

df.to_excel('df.xlsx', index=False)